# Labolatorium 5

Celem zadania jest opracowanie modelu klasyfikacji obrazów opartego na CNN, z uwzględnieniem
zoptymalizowanych parametrów augmentacji danych, wykorzystując zbiór danych CIFAR-10.
Kroki do wykonania:

1. Zaimportuj dane CIFAR-10. Podziel dane na X_train, y_train, X_test i y_test.
Protip: Użyj “from keras.datasets import cifar10”, a następnie

In [ ]:
import numpy as np
import os
import pickle
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import Adam

# Nie działały mi certifykaty SSL, więc pobrałem pliki

def load_batch(file_path):
    with open(file_path, 'rb') as file:
        data_dict = pickle.load(file, encoding='bytes')
        images = data_dict[b'data']
        labels = data_dict[b'labels']
        # Przekształcenie obrazów na odpowiedni kształt
        images = images.reshape(len(images), 3, 32, 32).transpose(0, 2, 3, 1)
        return images, np.array(labels)

def load_cifar10(data_dir):
    X_train, y_train = [], []
    for i in range(1, 6):
        file_path = os.path.join(data_dir, f"data_batch_{i}")
        images, labels = load_batch(file_path)
        X_train.append(images)
        y_train.append(labels)

    X_train = np.concatenate(X_train)
    y_train = np.concatenate(y_train)

    test_file = os.path.join(data_dir, "test_batch")
    X_test, y_test = load_batch(test_file)

    return (X_train, y_train), (X_test, y_test)

(X_train, y_train), (X_test, y_test) = load_cifar10('cifar-10-batches-py')


2. Ogranicz dane wejściowe w celu szybszej optymalizacji – na przykład do 3 lub 4
klas oraz 2 tys. zdjęć.

In [ ]:
selected_classes = [0, 1, 2, 3]
mask_train = np.isin(y_train, selected_classes)
mask_test = np.isin(y_test, selected_classes)

X_train, y_train = X_train[mask_train.squeeze()], y_train[mask_train.squeeze()]
X_test, y_test = X_test[mask_test.squeeze()], y_test[mask_test.squeeze()]

y_train = np.array([selected_classes.index(y) for y in y_train]).reshape(-1, 1)
y_test = np.array([selected_classes.index(y) for y in y_test]).reshape(-1, 1)

# Zmniejszenie zbioru treningowego
data_limit = 2000
X_train, y_train = X_train[:data_limit], y_train[:data_limit]

3. Dokonaj normalizacji danych wejściowych oraz etykiety przekształć na reprezentację
one-hot.

    Protip: Normalizacja to odjęcie średniej i podzielenie przez odchylenie std.
    Przekształcenie do one-hot za pomocą “keras.utils.to_categorical()”.

In [19]:
X_train = (X_train - X_train.mean()) / X_train.std()
X_test = (X_test - X_test.mean()) / X_test.std()

y_train = to_categorical(y_train, num_classes=len(selected_classes))
y_test = to_categorical(y_test, num_classes=len(selected_classes))

4. Zdefiniuj sieć CNN odpowiednią dla danych CIFAR.

In [20]:
def build_cnn():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=X_train.shape[1:]),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(len(selected_classes), activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

5. Przetrenuj model.

    Protip: Wyodrębnij zbiór walidacyjny ze zbioru treningowego, zdefiniuj
    batch_size i ilość epok.

In [21]:
batch_size = 32
epochs = 20

6. Zdefiniuj funkcję do augmentacji danych, uwzględniając transformacje
geometryczne, takie jak obroty, przesunięcia w pionie i poziomie oraz skalowanie.
Dokonaj parametryzacji tych transformacji, określając zakresy dla obrotów,
przesunięć i skalowania.

    Protip: Użyj “keras.preprocessing.image.ImageDataGenerator()”. Parametry:
rotation_range, width_shift_range, height_shift_range, zoom_range.

In [22]:
def create_data_generator(rotation_range, width_shift_range, height_shift_range, zoom_range):
    return ImageDataGenerator(
        rotation_range=rotation_range,
        width_shift_range=width_shift_range,
        height_shift_range=height_shift_range,
        zoom_range=zoom_range
    )

7. Potraktuj zakresy transformacji geometrycznych jako hiperparametry modelu i
przeprowadź ich dobór (np. przy użyciu podejścia Random Search). Jako kryterium
optymalizacji wykorzystaj dokładność uzyskaną w wyniku kroswalidacji k-fold.
Protip: Wykorzystaj “sklearn.model_selection.KFold”.

In [23]:
rotation_range_values = [10, 20, 30]
width_shift_values = [0.1, 0.2, 0.3]
height_shift_values = [0.1, 0.2, 0.3]
zoom_range_values = [0.1, 0.2, 0.3]

best_accuracy = 0
best_params = {}

kf = KFold(n_splits=3, shuffle=True, random_state=42)

for rotation_range in rotation_range_values:
    for width_shift_range in width_shift_values:
        for height_shift_range in height_shift_values:
            for zoom_range in zoom_range_values:
                val_accuracies = []
                data_gen = create_data_generator(rotation_range, width_shift_range, height_shift_range, zoom_range)

                for train_idx, val_idx in kf.split(X_train):
                    X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
                    y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]

                    model = build_cnn()
                    train_generator = data_gen.flow(X_train_fold, y_train_fold, batch_size=batch_size)
                    model.fit(train_generator, epochs=epochs, validation_data=(X_val_fold, y_val_fold), verbose=0)

                    val_loss, val_acc = model.evaluate(X_val_fold, y_val_fold, verbose=0)
                    val_accuracies.append(val_acc)

                mean_accuracy = np.mean(val_accuracies)

                if mean_accuracy > best_accuracy:
                    best_accuracy = mean_accuracy
                    best_params = {
                        'rotation_range': rotation_range,
                        'width_shift_range': width_shift_range,
                        'height_shift_range': height_shift_range,
                        'zoom_range': zoom_range
                    }

/Users/krzysztofbabicki/Documents/Edukacja/WSEI/WSEI - Semestr I/Zaawansowane metody sztucznej inteligencji/Laby/.venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/krzysztofbabicki/Documents/Edukacja/WSEI/WSEI - Semestr I/Zaawansowane metody sztucznej inteligencji/Laby/.venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8. Po określeniu optymalnych wartości parametrów przeprowadź końcowy trening na
pełnym zbiorze treningowym i ewaluacje na zbiorze testowym.

In [30]:
from sklearn.model_selection import train_test_split

final_data_gen = create_data_generator(**best_params)
final_model = build_cnn()

X_train_final, X_val_final, y_train_final, y_val_final = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

train_generator = final_data_gen.flow(X_train_final, y_train_final, batch_size=batch_size)
val_generator = final_data_gen.flow(X_val_final, y_val_final, batch_size=batch_size)

final_model.fit(train_generator, epochs=epochs, validation_data=val_generator, verbose=1)

test_loss, test_acc = final_model.evaluate(X_test, y_test, verbose=1)

Epoch 1/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.3925 - loss: 1.3288 - val_accuracy: 0.6050 - val_loss: 1.0512
Epoch 2/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5738 - loss: 1.0522 - val_accuracy: 0.6500 - val_loss: 0.8895
Epoch 3/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6412 - loss: 0.9087 - val_accuracy: 0.7100 - val_loss: 0.7688
Epoch 4/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6698 - loss: 0.8366 - val_accuracy: 0.7000 - val_loss: 0.7478
Epoch 5/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7012 - loss: 0.7665 - val_accuracy: 0.6700 - val_loss: 0.8344
Epoch 6/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6828 - loss: 0.7940 - val_accuracy: 0.7350 - val_loss: 0.7313
Epoch 7/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7058 - loss: 0.7389 - val_accuracy: 0.7600 - val_loss: 0.6279
Epoch 8/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7493 - loss: 0.6805 - val_accuracy: 0.7500 - val_loss

In [29]:
print("Best Hyperparameters:", best_params)
print("Test Accuracy:", test_acc)

Best Hyperparameters: {'rotation_range': 20, 'width_shift_range': 0.1, 'height_shift_range': 0.1, 'zoom_range': 0.1}
Test Accuracy: 0.7789999842643738
